In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)

In [2]:
df_full = pd.read_csv(r'./flatforsaleindhaka_spider.csv')

In [3]:
df1 = df_full.copy()

In [4]:
df1.head(3)

,baths,beds,location,parking,price,price_per_sqft,property_type,property_url,sqft
0,NaN,NaN,"Dhaka, Motijheel, Old Town",NaN,"2,000,000 BDT","20,000.00 BDT",Commercial,https://flatforsaleindhaka.com/listings/ready-...,100
1,3,3,"Dhaka, Green Road",1.0,"6,900,000 BDT","6,000.00 BDT",Apartment,https://flatforsaleindhaka.com/listings/ready-...,"1,150"
2,2,2,"Dhaka, Baridhara, Bashundhara, Kalachadpur",1.0,"3,357,500 BDT","3,950.00 BDT",Apartment,https://flatforsaleindhaka.com/listings/%e0%a6...,850


In [5]:
df1.shape

(21, 9)

In [6]:
# @Ekoue
df1["property_type"].unique()

array(['Commercial', 'Apartment', 'Apartment, Commercial'], dtype=object)

In [7]:
# @Ekoue
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   baths           15 non-null     object 
 1   beds            16 non-null     object 
 2   location        21 non-null     object 
 3   parking         17 non-null     float64
 4   price           21 non-null     object 
 5   price_per_sqft  21 non-null     object 
 6   property_type   21 non-null     object 
 7   property_url    21 non-null     object 
 8   sqft            21 non-null     object 
dtypes: float64(1), object(8)
memory usage: 1.6+ KB


In [8]:
df_fin = pd.DataFrame()

In [9]:
#location
# df_fin['city'] = df1.location.apply(lambda x: x.split(', ')[0])
# df_fin['district'] = df1.location.apply(lambda x: x.split(', ')[1])
# df_fin['otherZoneArea'] = df1.location.apply(lambda x: x.split(', ')[2:][0] if len(x.split(', ')[2:])>0 else '')

df_fin['city'] = df1.location.apply(lambda x: x.split(', ')[0])
df_fin['address'] = df1.location.apply(lambda x: ', '.join(x.split(', ')[1:]) if len(x.split(', '))>0 else '')

In [10]:
#area - to be decimal and not string 
df_fin['area'] = df1.sqft.apply(lambda x: x.replace(',',''))
df_fin.area = df_fin.area.astype(float)
df_fin.area.dtypes

dtype('float64')

In [11]:
#building_type
df_fin['building_type']  = df1.property_type
df_fin.building_type.unique()

array(['Commercial', 'Apartment', 'Apartment, Commercial'], dtype=object)

In [12]:
df_fin.building_type = df_fin.building_type.replace('Commercial', 'Commerical - Other')
#df_fin.building_type = df_fin.building_type.replace('Apartment, Commercial', 'Commerical - Other')
df_fin.building_type.unique()

array(['Commerical - Other', 'Apartment', 'Apartment, Commercial'],
      dtype=object)

In [13]:
# @Ekoue
"""
TODO

df_fin.building_type should contain informations like : 'Apartment', 'Office', 'Floor', 'Building', 'Plot', 'Shop',
       'Duplex', 'Warehouse', 'Factory', ...
       
'Commercial', 'Apartment' should be values found in df_fin.building_nature.
       
Could you set df_fin.building_type to NaN if you don't have the relevant information from the raw data ? It will allow 
   consistancy with the work of other collaborators.
"""

"\nTODO\n\ndf_fin.building_type should contain informations like : 'Apartment', 'Office', 'Floor', 'Building', 'Plot', 'Shop',\n       'Duplex', 'Warehouse', 'Factory', ...\n       \n'Commercial', 'Apartment' should be values found in df_fin.building_nature.\n       \nCould you set df_fin.building_type to NaN if you don't have the relevant information from the raw data ? It will allow \n   consistancy with the work of other collaborators.\n"

In [14]:
df1.property_type.unique()

array(['Commercial', 'Apartment', 'Apartment, Commercial'], dtype=object)

In [15]:
#building_nature
df_fin['building_nature'] = df1.property_type
df_fin.building_nature = df_fin.building_nature.replace('Apartment','Residential')
df_fin.building_nature.unique()

array(['Commercial', 'Residential', 'Apartment, Commercial'], dtype=object)

In [16]:
#num of bath and bed rooms
df_fin['num_bath_rooms'] = df1.baths
df_fin['num_bed_rooms'] = df1.beds

#fill na with 0
df_fin.num_bath_rooms.fillna(0,inplace=True)
df_fin.num_bed_rooms.fillna(0,inplace=True)

df_fin.num_bath_rooms.isna().sum(), df_fin.num_bed_rooms.isna().sum()

(0, 0)

In [17]:
# @Ekoue
df1['price'].unique()

array(['2,000,000 BDT', '6,900,000 BDT', '3,357,500 BDT',
       '22,525,000 BDT', '302,240,000 BDT', '3,280,000 BDT',
       '18,400,000 BDT', '16,000,000 BDT', '3,640,000 BDT',
       '6,322,500 BDT', '45,523,500 BDT', '15,360,000 BDT',
       '8,500,000 BDT', '9,736,000 BDT', '11,550,000 BDT',
       '12,000,000 BDT', '21,000,000 BDT', '15,184,000 BDT',
       '4,500,000 BDT', '8,220,000 BDT', '16,695,000 BDT'], dtype=object)

In [18]:
#price
df_fin['price'] = df1.price.apply(lambda x: x.split(' BDT')[0].replace(',','') )
df_fin.price = df_fin.price.astype(float)
df_fin.price.dtype

dtype('float64')

In [19]:
# @Ekoue
"""
TODO

    There is a df1.price_per_sqft column; this suggest that df1.price is the price for a part of the surface, not 
        the whole area.
    df_fin.price should be something like df_fin.price * df1.price_per_sqft 
"""

'\nTODO\n\n    There is a df1.price_per_sqft column; this suggest that df1.price is the price for a part of the surface, not \n        the whole area.\n    df_fin.price should be something like df_fin.price * df1.price_per_sqft \n'

In [20]:
#purpose
#since the data is not available, and the website mentions the sale of property, defaulting value to sale
df_fin['purpose'] = 'Sale'

In [21]:
#parking
df_fin['parking-spaces-amenity'] = df1.parking
df_fin['parking-spaces-amenity'].fillna(0,inplace=True)

In [22]:
df_fin['property_description'] = np.NaN
df_fin['property_overview'] = np.NAN

In [23]:
# @Ekoue
"""
TODO: 

    property_description and property_overview should have NaN (not '') as default value in order 
      to be exploited effeciently in EDA
"""

"\nTODO: \n\n    property_description and property_overview should have NaN (not '') as default value in order \n      to be exploited effeciently in EDA\n"

In [24]:
df_fin.head(6).T

,0,1,2,3,4,5
city,Dhaka,Dhaka,Dhaka,Dhaka,Dhaka,Dhaka
address,"Motijheel, Old Town",Green Road,"Baridhara, Bashundhara, Kalachadpur","Motijheel, Paltan, Segun Bagicha, Shantinagar,...",Gulshan,"Dhanmondi, Old Town"
area,100.0,1150.0,850.0,2650.0,7556.0,820.0
building_type,Commerical - Other,Apartment,Apartment,Apartment,Commerical - Other,"Apartment, Commercial"
building_nature,Commercial,Residential,Residential,Residential,Commercial,"Apartment, Commercial"
num_bath_rooms,0,3,2,"3, 4",0,0
num_bed_rooms,0,3,2,"3, 4",0,0
price,2000000.0,6900000.0,3357500.0,22525000.0,302240000.0,3280000.0
purpose,Sale,Sale,Sale,Sale,Sale,Sale
parking-spaces-amenity,0.0,1.0,1.0,1.0,2.0,1.0


In [25]:
df_fin.shape

(21, 12)

In [26]:
# @Ekoue
df_fin["num_bath_rooms"].unique()

array([0, '3', '2', '3, 4', '5', '4'], dtype=object)

In [27]:
# @Ekoue
df_fin["num_bed_rooms"].unique()

array([0, '3', '2', '3, 4', '4'], dtype=object)

In [28]:
df3 = df_fin.copy()
flatforsaleindhaka = (
    df3
    .assign(
        num_bath_rooms = lambda x: x["num_bath_rooms"].str.split(", "),
        num_bed_rooms = lambda x: x["num_bed_rooms"].str.split(", "),
        building_nature = lambda x: x["building_nature"].str.split(", ")
    )
    .explode("num_bath_rooms")
    .explode("num_bed_rooms")
    .explode("building_nature")
)

In [29]:
flatforsaleindhaka

,city,address,area,building_type,building_nature,num_bath_rooms,num_bed_rooms,price,purpose,parking-spaces-amenity,property_description,property_overview
0,Dhaka,"Motijheel, Old Town",100.0,Commerical - Other,Commercial,NaN,NaN,2000000.0,Sale,0.0,NaN,NaN
1,Dhaka,Green Road,1150.0,Apartment,Residential,3,3,6900000.0,Sale,1.0,NaN,NaN
2,Dhaka,"Baridhara, Bashundhara, Kalachadpur",850.0,Apartment,Residential,2,2,3357500.0,Sale,1.0,NaN,NaN
3,Dhaka,"Motijheel, Paltan, Segun Bagicha, Shantinagar,...",2650.0,Apartment,Residential,3,3,22525000.0,Sale,1.0,NaN,NaN
3,Dhaka,"Motijheel, Paltan, Segun Bagicha, Shantinagar,...",2650.0,Apartment,Residential,3,4,22525000.0,Sale,1.0,NaN,NaN
3,Dhaka,"Motijheel, Paltan, Segun Bagicha, Shantinagar,...",2650.0,Apartment,Residential,4,3,22525000.0,Sale,1.0,NaN,NaN
3,Dhaka,"Motijheel, Paltan, Segun Bagicha, Shantinagar,...",2650.0,Apartment,Residential,4,4,22525000.0,Sale,1.0,NaN,NaN
4,Dhaka,Gulshan,7556.0,Commerical - Other,Commercial,NaN,NaN,302240000.0,Sale,2.0,NaN,NaN
5,Dhaka,"Dhanmondi, Old Town",820.0,"Apartment, Commercial",Apartment,NaN,NaN,3280000.0,Sale,1.0,NaN,NaN
5,Dhaka,"Dhanmondi, Old Town",820.0,"Apartment, Commercial",Commercial,NaN,NaN,3280000.0,Sale,1.0,NaN,NaN


In [30]:
flatforsaleindhaka.num_bath_rooms.fillna(0,inplace=True)
flatforsaleindhaka.num_bed_rooms.fillna(0,inplace=True)

In [31]:
flatforsaleindhaka.num_bath_rooms = flatforsaleindhaka.num_bath_rooms.astype(float)
flatforsaleindhaka.num_bed_rooms = flatforsaleindhaka.num_bed_rooms.astype(float)

In [32]:
flatforsaleindhaka.shape

(25, 12)

In [33]:
df_fin

,city,address,area,building_type,building_nature,num_bath_rooms,num_bed_rooms,price,purpose,parking-spaces-amenity,property_description,property_overview
0,Dhaka,"Motijheel, Old Town",100.0,Commerical - Other,Commercial,0,0,2000000.0,Sale,0.0,NaN,NaN
1,Dhaka,Green Road,1150.0,Apartment,Residential,3,3,6900000.0,Sale,1.0,NaN,NaN
2,Dhaka,"Baridhara, Bashundhara, Kalachadpur",850.0,Apartment,Residential,2,2,3357500.0,Sale,1.0,NaN,NaN
3,Dhaka,"Motijheel, Paltan, Segun Bagicha, Shantinagar,...",2650.0,Apartment,Residential,"3, 4","3, 4",22525000.0,Sale,1.0,NaN,NaN
4,Dhaka,Gulshan,7556.0,Commerical - Other,Commercial,0,0,302240000.0,Sale,2.0,NaN,NaN
5,Dhaka,"Dhanmondi, Old Town",820.0,"Apartment, Commercial","Apartment, Commercial",0,0,3280000.0,Sale,1.0,NaN,NaN
6,Dhaka,"Motijheel, Paltan, Segun Bagicha, Shantinagar",1150.0,Commerical - Other,Commercial,0,0,18400000.0,Sale,1.0,NaN,NaN
7,Dhaka,"Segun Bagicha, Shantinagar, Siddeshwari",1590.0,Apartment,Residential,3,3,16000000.0,Sale,1.0,NaN,NaN
8,Dhaka,"Dhanmondi, Mohammadpur",91.0,Commerical - Other,Commercial,0,0,3640000.0,Sale,0.0,NaN,NaN
9,Dhaka,Mirpur,1405.0,Apartment,Residential,3,3,6322500.0,Sale,1.0,NaN,NaN


In [34]:
# @Ekoue
flatforsaleindhaka

,city,address,area,building_type,building_nature,num_bath_rooms,num_bed_rooms,price,purpose,parking-spaces-amenity,property_description,property_overview
0,Dhaka,"Motijheel, Old Town",100.0,Commerical - Other,Commercial,0.0,0.0,2000000.0,Sale,0.0,NaN,NaN
1,Dhaka,Green Road,1150.0,Apartment,Residential,3.0,3.0,6900000.0,Sale,1.0,NaN,NaN
2,Dhaka,"Baridhara, Bashundhara, Kalachadpur",850.0,Apartment,Residential,2.0,2.0,3357500.0,Sale,1.0,NaN,NaN
3,Dhaka,"Motijheel, Paltan, Segun Bagicha, Shantinagar,...",2650.0,Apartment,Residential,3.0,3.0,22525000.0,Sale,1.0,NaN,NaN
3,Dhaka,"Motijheel, Paltan, Segun Bagicha, Shantinagar,...",2650.0,Apartment,Residential,3.0,4.0,22525000.0,Sale,1.0,NaN,NaN
3,Dhaka,"Motijheel, Paltan, Segun Bagicha, Shantinagar,...",2650.0,Apartment,Residential,4.0,3.0,22525000.0,Sale,1.0,NaN,NaN
3,Dhaka,"Motijheel, Paltan, Segun Bagicha, Shantinagar,...",2650.0,Apartment,Residential,4.0,4.0,22525000.0,Sale,1.0,NaN,NaN
4,Dhaka,Gulshan,7556.0,Commerical - Other,Commercial,0.0,0.0,302240000.0,Sale,2.0,NaN,NaN
5,Dhaka,"Dhanmondi, Old Town",820.0,"Apartment, Commercial",Apartment,0.0,0.0,3280000.0,Sale,1.0,NaN,NaN
5,Dhaka,"Dhanmondi, Old Town",820.0,"Apartment, Commercial",Commercial,0.0,0.0,3280000.0,Sale,1.0,NaN,NaN


In [35]:
flatforsaleindhaka.to_csv(r'./cleaned_flatforsaleindhaka_spider.csv',index=False)

In [36]:
df1.head(6)

,baths,beds,location,parking,price,price_per_sqft,property_type,property_url,sqft
0,NaN,NaN,"Dhaka, Motijheel, Old Town",NaN,"2,000,000 BDT","20,000.00 BDT",Commercial,https://flatforsaleindhaka.com/listings/ready-...,100
1,3,3,"Dhaka, Green Road",1.0,"6,900,000 BDT","6,000.00 BDT",Apartment,https://flatforsaleindhaka.com/listings/ready-...,"1,150"
2,2,2,"Dhaka, Baridhara, Bashundhara, Kalachadpur",1.0,"3,357,500 BDT","3,950.00 BDT",Apartment,https://flatforsaleindhaka.com/listings/%e0%a6...,850
3,"3, 4","3, 4","Dhaka, Motijheel, Paltan, Segun Bagicha, Shant...",1.0,"22,525,000 BDT","8,500.00 BDT",Apartment,https://flatforsaleindhaka.com/listings/flat-s...,"2,650"
4,NaN,NaN,"Dhaka, Gulshan",2.0,"302,240,000 BDT","40,000.00 BDT",Commercial,https://flatforsaleindhaka.com/listings/office...,"7,556"
5,NaN,NaN,"Dhaka, Dhanmondi, Old Town",1.0,"3,280,000 BDT","4,000.00 BDT","Apartment, Commercial",https://flatforsaleindhaka.com/listings/flats-...,820


In [37]:
flatforsaleindhaka.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 20
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    25 non-null     object 
 1   address                 25 non-null     object 
 2   area                    25 non-null     float64
 3   building_type           25 non-null     object 
 4   building_nature         25 non-null     object 
 5   num_bath_rooms          25 non-null     float64
 6   num_bed_rooms           25 non-null     float64
 7   price                   25 non-null     float64
 8   purpose                 25 non-null     object 
 9   parking-spaces-amenity  25 non-null     float64
 10  property_description    0 non-null      float64
 11  property_overview       0 non-null      float64
dtypes: float64(7), object(5)
memory usage: 2.5+ KB
